In [ ]:
import os, subprocess
%cd '/home/cslage/Research/LSST/code/covariance2'
command = 'dm_setup.sh'
dm = subprocess.Popen(command, shell=True)
subprocess.Popen.wait(dm)
#subprocess.call('./dm_setup.sh')
from lsst.eotest.sensor.MaskedCCD import MaskedCCD

In [ ]:
os.environ['LSST_HOME']='/sandbox/lsstshared/v10_0'
%setup anaconda

%cd ..
%env LSST_HOME='/sandbox/lsstshared/v10_0'

%setup -k -r ../covariance2/Linux64/pipe_tasks/
%setup -k -r ../covariance2/eotest/eotest-0.0.0.7/


In [ ]:
#import matplotlib
#matplotlib.use("PDF")
from pylab import *
import astropy.io.fits as pf

import os, sys, glob, time, scipy
from scipy import stats
import scipy.interpolate
from subprocess import call
from itertools import cycle
import pickle
from lsst.eotest.sensor.MaskedCCD import MaskedCCD
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath
import lsst.afw.geom as Geom
import numpy as np

#sys.path.append('/home/cslage/Software/forward_model_convolved')
#import  forward_convolved


In [ ]:
stormdir = "/mnt/storm/GUI/"
files1 = np.sort(glob.glob(stormdir+'20180516_002_flats_2/ITL-3800C-002_flat_flat_?00_20180516??????.fits'))
files2 = np.sort(glob.glob(stormdir+'20180516_002_flats_2/ITL-3800C-002_flat_flat_?01_20180516??????.fits'))

print(len(files1),len(files2))


In [ ]:
def GetRegions(regfilename):
    file=open(regfilename)
    lines=file.readlines()
    file.close()
    regions=[]
    for line in lines[3:]:
        if line[:6]=='# tile':
            tilenum="%02d"%(int(line.split()[2]))
        if line[:3]=='box':
            regnums=line.strip('box()\n').split(',')
            yc,xc,ys,xs=[int(float(val)) for val in regnums[0:4]]
            regions.append([xc,yc,xs,ys,tilenum])
    return regions

regfilename=stormdir+'notebooks/focuscurve_vs_flux_1.reg'
regions = GetRegions(regfilename)
numregions = len(regions)
print(numregions)

In [ ]:
mask_grow = 5
numsegments = 16
regionnumber = 0
reg = regions[regionnumber]
xcen,ycen,xsize,ysize,tilenum=reg
for n in range(len(files1)):
    ccd1=MaskedCCD(files1[n])
    ccd2=MaskedCCD(files2[n])

    if ccd1.md.get('EXPTIME') != ccd2.md.get('EXPTIME'):
        raise RuntimeError("Exposure times for files %s, %s do not match"%(file1, file2))

    for segment in [0]:#range(numsegments):
        time_start=time.time()
        amp=segment + 1
        extname=pf.getheader(patterns[n],amp)['EXTNAME'] 
        if extname in []: # Skip bad segments
            continue
        print(extname)
        image1 = ccd1.unbiased_and_trimmed_image(amp)
        image2 = ccd2.unbiased_and_trimmed_image(amp)
        ccddata1 = image1.getArrays()[0][xcen-xsize/2:xcen+xsize/2,ycen-ysize/2:ycen+ysize/2]
        ccddata2 = image2.getArrays()[0][xcen-xsize/2:xcen+xsize/2,ycen-ysize/2:ycen+ysize/2]
        print(ccddata1.shape)
        med1 = np.median(ccddata1)
        med2 = np.median(ccddata2)
        avemed  = (med1 + med2) / 2.0
        # Now mask out areas in the two images where the image value exceeds median +/-N sigma
        # Assume the variance = median, as it should be for a flat field
        Nsigma=6
        var1 = med1
        thresh1 = Nsigma*sqrt(var1)
        mask1 = np.ma.masked_where((abs(ccddata1 - med1) > thresh1),ccddata1 ,copy=True)
        print("Mask1 masking out %d pixels"%mask1.mask.sum())
        var2 = med2
        thresh2 = Nsigma*sqrt(var2)
        mask2 = np.ma.masked_where((abs(ccddata2 - med2) > thresh2),ccddata2 ,copy=True)
        print("Mask2 masking out %d pixels"%mask2.mask.sum())
        #subtract the two images
        fdiff = mask1
        fdiff -= mask2
        # Expand mask region around bad pixels:
        expanded_mask = np.zeros([ysize, xsize],dtype=bool)
        for i in range(ysize):
            for j in range(xsize):
                if fdiff.mask[i,j]:
                    for ii in range(i - mask_grow, i + mask_grow+1):
                        if ii < 0 or ii > ysize - 1:
                            continue
                        for jj in range(j - mask_grow, j + mask_grow+1):
                            if jj <0 or jj > xsize - 1:
                                continue
                            expanded_mask[ii,jj] = True
        # Add the edge mask
        fdiff.mask=np.logical_or(expanded_mask,edges)
        (nrows,ncols)=fdiff.shape
        print("Diff Masking out %d pixels, including edges"%fdiff.mask.sum())
        print("For file %d, segment = %s,image1 median = %.2f, image2 median = %.2f"%(n,extname, median(ccddata1), median(ccddata2)))
        print("Diff min = %f, max = %f, mean = %f, median = %f"%(np.ma.min(fdiff), np.ma.max(fdiff), np.ma.mean(fdiff), np.ma.median(fdiff)))
        # Finally compute the pixel covariance
        k=1
        l=1
        i_range = range(k,ncols)
        j_range = range(l,nrows)

        for k in range(0,6):
            for l in range(0,6):
                npixused1=0
                if k==0 and l==0:
                    corr = np.ma.var(fdiff)
                else:
                    temp1=fdiff[l:nrows  ,   k:ncols]
                    data1=temp1.data
                    mask1=temp1.mask
                    temp2=fdiff[0:nrows-l  , 0:ncols-k]
                    data2=temp2.data
                    mask2=temp2.mask
                    or_mask=np.logical_or(mask1,mask2)
                    npixused1=or_mask.size-or_mask.sum()
                    sub1=np.ma.MaskedArray(data1,or_mask)
                    sub2=np.ma.MaskedArray(data2,or_mask)
                    sum11=sub1.sum()
                    sum21=sub2.sum()
                    sum121=(sub1*sub2).sum()
                    corr = (sum121 - sum11*sum21/npixused1)/npixused1
                
                print("For file %d, segment = %s,NumDataPoints = %d, ii = %d, jj = %d,Cij = %.2f"%(n, extname, npixused1, l, k, corr))
                #file = open(filename,'a')
                if k==0 and l==0:
                    line = '%d     %d     %d     %s     %f      %f\n'%(l,k,n,extname,corr,avemed)
                else:
                    line = '%d     %d     %d     %s     %f\n'%(l,k,n,extname,corr)
                print(line)#file.write(line)
                #file.close()
    time_finish = time.time()
    elapsed = time_finish - time_start
    print("Elapsed time for file %d  = %.2f"%(n,elapsed))
    
